In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from textblob import TextBlob
import re

driver = webdriver.Chrome()
driver.get("https://twitter.com/login")

In [2]:
username = driver.find_element(By.XPATH,"//input[@name='text']")
username.send_keys('alcohold315@gmail.com') # f12.2021.00050 Albert Zefanya U
next_button = driver.find_element(By.XPATH,"//span[contains(text(),'Next')]")
next_button.click()

In [3]:
password = driver.find_element(By.XPATH, "//input[@name='password']")
password.send_keys('b64e11r7t')
password.send_keys(Keys.RETURN)

In [4]:
sleep(3)
search_box = driver.find_element(By.XPATH, "//input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys("bakso bakar")
search_box.send_keys(Keys.ENTER)

In [5]:
UserTags = []
TimeStamps = []
Tweets = []
Replys = [] 
reTweets = []
Likes = []
Views = []
Sentimen = []

articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
while True:
    articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")

    for article in articles:
        try:
            # Wait for the user tag element to be present before trying to find it
            user_tag_element = WebDriverWait(article, 10).until(
                EC.presence_of_element_located((By.XPATH, ".//div[@data-testid='cellInnerDiv']"and".//div[@data-testid='User-Name']"and".//span[contains(text(), '@')]"))
            )
            
            UserTag = user_tag_element.text
        except selenium.common.exceptions.TimeoutException:
            # Handle timeout exception
            UserTag = ""

        UserTags.append(UserTag)


        TimeStamp = article.find_element(By.XPATH, ".//div[@data-testid='cellInnerDiv']"and".//div[@data-testid='User-Name']"and".//time").get_attribute('datetime')
        TimeStamps.append(TimeStamp)
                                          
        try:
            Tweet = article.find_element(By.XPATH,".//div[@data-testid='tweetText']").text
        except selenium.common.exceptions.NoSuchElementException:
            Tweet = ""
        Tweets.append(Tweet)

        Reply = article.find_element(By.XPATH,".//div[@data-testid='reply']").text
        Replys.append(Reply)

        reTweet = article.find_element(By.XPATH,".//div[@data-testid='retweet']").text
        reTweets.append(reTweet)

        Like = article.find_element(By.XPATH,".//div[@data-testid='like']").text
        Likes.append(Like)

        try:
            View = article.find_element(By.XPATH,".//div[@data-testid='cellInnerDiv']"and".//div[@role='group']"and".//a[contains(@aria-label, '"+"views"+"')]").text
        except selenium.common.exceptions.NoSuchElementException as e:
            View = ""
        Views.append(View)

        clean_tweet = ' '.join(re.sub(r"(@[A-Za-z0-9]+)|(^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",Tweet).split())
        analysis = TextBlob(clean_tweet)
        if analysis.sentiment.polarity > 0.0:
            sentiment = '1.0'
        elif analysis.sentiment.polarity == 0.0:
            sentiment = '0.0'
        else:
            sentiment = '-1.0'
        Sentimen.append(sentiment)

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    sleep(3)
    articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
    Tweets2 = list(set(Tweets))
    if len(Tweets2) > 50:
        break

In [9]:
import pandas as pd
from time import sleep
import os
import subprocess

# Assuming UserTags, TimeStamps, etc., are defined somewhere in your code
# ...

df = pd.DataFrame(
    zip(UserTags, TimeStamps, Tweets, Replys, reTweets, Likes, Views, Sentimen),
    columns=['UserTags', 'TimeStamps', 'Tweets', 'Replys', 'reTweets', 'Likes', 'Views', 'Sentimen']
)

excel_file_path = "D:\\tweets_live.xlsx"

try:
    df.to_excel(excel_file_path, index=False)
    sleep(3)
    
    # Use subprocess to open the file with the default application
    subprocess.Popen(['start', 'excel', excel_file_path], shell=True)
except Exception as e:
    print(f"Error: {e}")
